<a href="https://colab.research.google.com/github/GreatLearningAIML1/bangalore-aug19-batch-vybhavsv/blob/master/R8_Statiscal_NLP_Project_Vybhav_SV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project description <br> 

Classification is probably the most popular task that you would deal with in real life. Text in the form of blogs, posts, articles, etc. is written every second. It is a challenge to predict the information about the writer without knowing about him/her. We are going to create a classifier that predicts multiple features of the author of a given text.We have designed it as a Multilabel classification problem.

# Dataset

Blog Authorship Corpus Over 600,000 posts from more than 19 thousand bloggers The Blog Authorship Corpus consists of the collected posts of 19,320 bloggers gathered from blogger.com in August 2004. The corpus incorporates a total of 681,288 posts and over 140 million words - or approximately 35 posts and 7250 words per person. Each blog is presented as a separate file, the name of which indicates a blogger id# and the blogger’s self-provided gender, age, industry, and astrological sign. (All are labeled for gender and age but for many, industry and/or sign is marked as unknown.) All bloggers included in the corpus fall into one of three age groups: 
*   8240 "10s" blogs (ages 13-17), 
*   8086 "20s" blogs(ages 23-27) 
*   2994 "30s" blogs (ages 33-47)

For each age group, there is an equal number of male and female bloggers. Each blog in the corpus includes at least 200 occurrences of common English words. All formatting has been stripped with two exceptions. Individual posts within a single blogger are separated by the date of the following post and links within a post are denoted by the label urllink.

Link to dataset: https://www.kaggle.com/rtatman/blog-authorship-corpus/downloads/blog-authorship-corpus.zip/2at

# Approach and Steps


1. Load the dataset (5 points)
a. Tip: As the dataset is large, use fewer rows. Check what is working well on your machine and decide accordingly.
2. Preprocess rows of the “text” column (7.5 points)
a. Remove unwanted characters
b. Convert text to lowercase
c. Remove unwanted spaces
d. Remove stopwords
3. As we want to make this into a multi-label classification problem, you are required to merge all the label columns together, so that we have all the labels together for a particular sentence (7.5 points)
a. Label columns to merge: “gender”, “age”, “topic”, “sign”
b. After completing the previous step, there should be only two columns in your data frame i.e. “text” and “labels” as shown in the below image
4. Separate features and labels, and split the data into training and testing (5 points)
5. Vectorize the features (5 points)
a. Create a Bag of Words using count vectorizer
i. Use ngram_range=(1, 2)
ii. Vectorize training and testing features
b. Print the term-document matrix
6. Create a dictionary to get the count of every label i.e. the key will be label name and value will be the total count of the label. Check below image for reference (5 points)
7. Transform the labels - (7.5 points) As we have noticed before, in this task each example can have multiple tags. To deal with such kind of prediction, we need to transform labels in a binary form and the prediction will be a mask of 0s and 1s. For this purpose, it is convenient to use MultiLabelBinarizer from sklearn
a. Convert your train and test labels using MultiLabelBinarizer
8. Choose a classifier - (5 points) In this task, we suggest using the One-vs-Rest approach, which is implemented in OneVsRestClassifier class. In this approach k classifiers (= number of tags) are trained. As a basic classifier, use LogisticRegression. It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time because the number of classifiers to train is large.
a. Use a linear classifier of your choice, wrap it up in OneVsRestClassifier to train it on every label
b. As One-vs-Rest approach might not have been discussed in the sessions, we are providing you the code for that
9. Fit the classifier, make predictions and get the accuracy (5 points)
a. Print the following
i. Accuracy score
ii. F1 score
iii. Average precision score
iv. Average recall score
v. Tip: Make sure you are familiar with all of them. How would you expect the things to work for the multi-label scenario? Read about micro/macro/weighted averaging
10. Print true label and predicted label for any five examples (7.5 points)

### 1. Mount the google drive, read the dataset (blog-authorship-corpus) and drop the NA's while reading the dataset

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd
import numpy as np
# read file into pandas using a relative path. Please change the path as needed
blogs_df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/blog-authorship-corpus.zip').dropna()

In [0]:
dataframe.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


#   1.b limiting dataframe to 4857 rows 

In [0]:
text_df=blogs_df.iloc[:4857,0:]

In [0]:
text_df

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
4852,1233335,male,25,BusinessServices,Sagittarius,"27,May,2004",There is khaos in my life... and now th...
4853,1233335,male,25,BusinessServices,Sagittarius,"26,May,2004","heather, i taped it for you. you're goi..."
4854,1233335,male,25,BusinessServices,Sagittarius,"25,May,2004",I made a doctor's appointment today to ...
4855,1233335,male,25,BusinessServices,Sagittarius,"25,May,2004","Well, just got back from the Skar and f..."


### 2. Preprocess NLP text (lowercase, remove special characters, remove numbers, remove emails, etc)

Defined a function preprocess, this will cater to lower case conversion, removal of html tags, url, stemming and lemmatization of the words.

In [0]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


text_df['text']=text_df['text'].map(lambda s:preprocess(s))

In [0]:
text_df

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004",info found pages pdf files wait untill team le...
1,2059027,male,15,Student,Leo,"13,May,2004",team members drewes van der laag urllink mail ...
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie aarde maak eigen water...
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoo toolbar capture urls popups means...
...,...,...,...,...,...,...,...
4852,1233335,male,25,BusinessServices,Sagittarius,"27,May,2004",khaos life also reason today mom called said e...
4853,1233335,male,25,BusinessServices,Sagittarius,"26,May,2004",heather taped going pee pants think season fin...
4854,1233335,male,25,BusinessServices,Sagittarius,"25,May,2004",made doctor appointment today deal shots need ...
4855,1233335,male,25,BusinessServices,Sagittarius,"25,May,2004",well got back skar found today sure going braz...


In [0]:
text_df.dtypes

id         int64
gender    object
age        int64
topic     object
sign      object
date      object
text      object
dtype: object

### 3. Create label column to merge: “gender”, “age”, “topic”, “sign”



In [0]:
text_df['label']=text_df['gender'].astype(str)+','+text_df['age'].astype(str)+','+text_df['topic'].astype(str)+','+text_df['sign'].astype(str)

In [0]:
text_df

,id,gender,age,topic,sign,date,text,label
0,2059027,male,15,Student,Leo,"14,May,2004",info found pages pdf files wait untill team le...,"male,15,Student,Leo"
1,2059027,male,15,Student,Leo,"13,May,2004",team members drewes van der laag urllink mail ...,"male,15,Student,Leo"
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie aarde maak eigen water...,"male,15,Student,Leo"
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing,"male,15,Student,Leo"
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoo toolbar capture urls popups means...,"male,33,InvestmentBanking,Aquarius"
...,...,...,...,...,...,...,...,...
4852,1233335,male,25,BusinessServices,Sagittarius,"27,May,2004",khaos life also reason today mom called said e...,"male,25,BusinessServices,Sagittarius"
4853,1233335,male,25,BusinessServices,Sagittarius,"26,May,2004",heather taped going pee pants think season fin...,"male,25,BusinessServices,Sagittarius"
4854,1233335,male,25,BusinessServices,Sagittarius,"25,May,2004",made doctor appointment today deal shots need ...,"male,25,BusinessServices,Sagittarius"
4855,1233335,male,25,BusinessServices,Sagittarius,"25,May,2004",well got back skar found today sure going braz...,"male,25,BusinessServices,Sagittarius"


In [0]:
text_df1 = text_df.iloc[:,6:8]

In [0]:
text_df1

,text,label
0,info found pages pdf files wait untill team le...,"male,15,Student,Leo"
1,team members drewes van der laag urllink mail ...,"male,15,Student,Leo"
2,het kader van kernfusie aarde maak eigen water...,"male,15,Student,Leo"
3,testing testing,"male,15,Student,Leo"
4,thanks yahoo toolbar capture urls popups means...,"male,33,InvestmentBanking,Aquarius"
...,...,...
4852,khaos life also reason today mom called said e...,"male,25,BusinessServices,Sagittarius"
4853,heather taped going pee pants think season fin...,"male,25,BusinessServices,Sagittarius"
4854,made doctor appointment today deal shots need ...,"male,25,BusinessServices,Sagittarius"
4855,well got back skar found today sure going braz...,"male,25,BusinessServices,Sagittarius"


Defined a function for converting label to string, this will help in identifying different labels contained in a same row

In [0]:
def convert2dict(sentence):
    sentence=str(sentence)
    return sentence.split(',')

text_df1['label1']=text_df1['label'].map(lambda s:convert2dict(s))

Maintaing 2 columns ( label and label1)

'label column'  : to maintain the count of dictionary label in step 6 

'label1 column' : populated after converting to string using comma  separated format

In [0]:
text_df1

,text,label,label1
0,info found pages pdf files wait untill team le...,"male,15,Student,Leo","[male, 15, Student, Leo]"
1,team members drewes van der laag urllink mail ...,"male,15,Student,Leo","[male, 15, Student, Leo]"
2,het kader van kernfusie aarde maak eigen water...,"male,15,Student,Leo","[male, 15, Student, Leo]"
3,testing testing,"male,15,Student,Leo","[male, 15, Student, Leo]"
4,thanks yahoo toolbar capture urls popups means...,"male,33,InvestmentBanking,Aquarius","[male, 33, InvestmentBanking, Aquarius]"
...,...,...,...
4852,khaos life also reason today mom called said e...,"male,25,BusinessServices,Sagittarius","[male, 25, BusinessServices, Sagittarius]"
4853,heather taped going pee pants think season fin...,"male,25,BusinessServices,Sagittarius","[male, 25, BusinessServices, Sagittarius]"
4854,made doctor appointment today deal shots need ...,"male,25,BusinessServices,Sagittarius","[male, 25, BusinessServices, Sagittarius]"
4855,well got back skar found today sure going braz...,"male,25,BusinessServices,Sagittarius","[male, 25, BusinessServices, Sagittarius]"


### 4. Separate features and labels, and split the data into training and testing (5 points)



In [0]:
from sklearn.model_selection import train_test_split

x = text_df1['text']
y = text_df1['label1']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=1)

In [0]:
print(x_train.shape)
print(x_test.shape)

(3399,)
(1458,)


In [0]:
print(y_train.shape)
print(y_test.shape)

(3399,)
(1458,)


In [0]:
y_train

1151                [female, 15, Student, Libra]
3854                 [male, 14, Student, Pisces]
1523               [male, 35, Technology, Aries]
545            [female, 27, Education, Aquarius]
2354               [male, 35, Technology, Aries]
                          ...                   
2895               [male, 35, Technology, Aries]
2763               [male, 35, Technology, Aries]
905     [male, 17, Sports-Recreation, Capricorn]
3980                     [male, 25, Arts, Aries]
235                [male, 15, Student, Aquarius]
Name: label1, Length: 3399, dtype: object

# 5. Vectorize the features

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1, 2))

# create document-term matrices
x_train_ct = cv.fit_transform(x_train)
x_test_ct = cv.transform(x_test)

In [0]:
#checking the vocabulary size
len(cv.vocabulary_)

234284

In [0]:
#what is there in vocabulary
cv.vocabulary_

{'goodness': 83141,
 'nothing': 138913,
 'whole': 226244,
 'day': 46878,
 'seriously': 178972,
 'went': 225092,
 'online': 142770,
 'hours': 94820,
 'called': 27503,
 'bob': 21919,
 'gentlemen': 78511,
 'found': 74220,
 'leaving': 110863,
 'little': 115648,
 'bit': 20140,
 'anywho': 9093,
 'stayed': 192237,
 'home': 93077,
 'dang': 46306,
 'fell': 69058,
 'asleep': 11821,
 'though': 204908,
 'around': 10516,
 'almost': 4599,
 'quite': 161086,
 'sleepy': 184932,
 'really': 163772,
 'hoping': 94157,
 'tomarrow': 208895,
 'better': 18982,
 'knows': 107609,
 'everyone': 63385,
 'going': 81673,
 'warped': 221753,
 'tour': 210103,
 'dad': 45635,
 'suck': 195847,
 'pits': 151700,
 'think': 203843,
 'meaning': 125075,
 'leave': 110689,
 'alone': 4762,
 'time': 206566,
 'boring': 22847,
 'maybe': 124424,
 'find': 70421,
 'someone': 186830,
 'tag': 198740,
 'along': 4853,
 'sarah': 173258,
 'supposed': 197066,
 'driving': 55580,
 'danceline': 46227,
 'marching': 123036,
 'band': 15520,
 'starts'

Building DTM and vectorizing train and test features

In [0]:
x_train_ct.shape

(3399, 234284)

In [0]:
print(x_train_ct[0])

  (0, 83141)	1
  (0, 138913)	1
  (0, 226244)	2
  (0, 46878)	3
  (0, 178972)	1
  (0, 225092)	1
  (0, 142770)	1
  (0, 94820)	1
  (0, 27503)	1
  (0, 21919)	1
  (0, 78511)	1
  (0, 74220)	2
  (0, 110863)	1
  (0, 115648)	1
  (0, 20140)	1
  (0, 9093)	1
  (0, 192237)	1
  (0, 93077)	1
  (0, 46306)	1
  (0, 69058)	1
  (0, 11821)	1
  (0, 204908)	1
  (0, 10516)	1
  (0, 4599)	1
  (0, 161086)	1
  :	:
  (0, 187038)	1
  (0, 198741)	1
  (0, 4951)	1
  (0, 173281)	1
  (0, 197096)	1
  (0, 74264)	1
  (0, 55594)	1
  (0, 46228)	1
  (0, 47062)	1
  (0, 81679)	1
  (0, 4806)	1
  (0, 123038)	1
  (0, 15573)	1
  (0, 191709)	1
  (0, 111651)	1
  (0, 130009)	1
  (0, 47495)	1
  (0, 64043)	1
  (0, 103937)	1
  (0, 86175)	1
  (0, 213883)	1
  (0, 209180)	1
  (0, 22832)	1
  (0, 57748)	1
  (0, 37875)	1


In [0]:
x_test_ct.shape

(1458, 234284)

In [0]:
x_test_ct[0]

<1x234284 sparse matrix of type '<class 'numpy.int64'>'
	with 199 stored elements in Compressed Sparse Row format>

### 6. Create a dictionary to get the count of every label

In [0]:
 text=[]
 text.append(df1['label'].str.cat(sep=','))

In [0]:
text

['male,15,Student,Leo,male,15,Student,Leo,male,15,Student,Leo,male,15,Student,Leo,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,InvestmentBanking,Aquarius,male,33,

In [0]:
def convert(lst): 
    return (lst[0].split(',')) 
  
# Driver code 
text = convert(text)
print(text)

['male', '15', 'Student', 'Leo', 'male', '15', 'Student', 'Leo', 'male', '15', 'Student', 'Leo', 'male', '15', 'Student', 'Leo', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'InvestmentBanking', 'Aquarius', 'male', '33', 'Investment

In [0]:
dictionary = {}
for label in text :
  dictionary[label] = dictionary.get(label, 0) + 1

dictionary

{'14': 170,
 '15': 339,
 '16': 67,
 '17': 215,
 '23': 137,
 '24': 353,
 '25': 241,
 '26': 96,
 '27': 86,
 '33': 101,
 '34': 540,
 '35': 2307,
 '36': 60,
 '37': 19,
 '39': 79,
 '41': 14,
 '42': 9,
 '44': 3,
 '45': 14,
 '46': 7,
 'Accounting': 2,
 'Aquarius': 329,
 'Aries': 2483,
 'Arts': 31,
 'Automotive': 14,
 'Banking': 16,
 'BusinessServices': 60,
 'Cancer': 94,
 'Capricorn': 84,
 'Communications-Media': 61,
 'Consulting': 16,
 'Education': 118,
 'Engineering': 119,
 'Gemini': 86,
 'Internet': 20,
 'InvestmentBanking': 70,
 'Law': 3,
 'Leo': 190,
 'Libra': 414,
 'Museums-Libraries': 2,
 'Non-Profit': 47,
 'Pisces': 67,
 'Religion': 4,
 'Sagittarius': 677,
 'Science': 33,
 'Scorpio': 292,
 'Sports-Recreation': 75,
 'Student': 569,
 'Taurus': 100,
 'Technology': 2332,
 'Virgo': 41,
 'female': 1590,
 'indUnk': 1265,
 'male': 3267}

### 7. Transform the labels
Using -  MultiLabelBinarizer from sklearn - transform labels in a binary form

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train_mlb=mlb.fit_transform(y_train)
y_test_mlb=mlb.transform(y_test)

In [0]:
print(y_train_mlb)
print(y_train_mlb.shape)

[[0 1 0 ... 1 0 0]
 [1 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 1 0 ... 0 0 1]]
(3399, 54)


In [0]:
print(y_test_mlb)
print(y_test_mlb.shape)

[[0 0 0 ... 1 0 0]
 [0 0 0 ... 1 1 0]
 [0 1 0 ... 1 0 0]
 ...
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 1]]
(1458, 54)


In [0]:
list(mlb.classes_)

['14',
 '15',
 '16',
 '17',
 '23',
 '24',
 '25',
 '26',
 '27',
 '33',
 '34',
 '35',
 '36',
 '37',
 '39',
 '41',
 '42',
 '44',
 '45',
 '46',
 'Accounting',
 'Aquarius',
 'Aries',
 'Arts',
 'Automotive',
 'Banking',
 'BusinessServices',
 'Cancer',
 'Capricorn',
 'Communications-Media',
 'Consulting',
 'Education',
 'Engineering',
 'Gemini',
 'Internet',
 'InvestmentBanking',
 'Law',
 'Leo',
 'Libra',
 'Museums-Libraries',
 'Non-Profit',
 'Pisces',
 'Religion',
 'Sagittarius',
 'Science',
 'Scorpio',
 'Sports-Recreation',
 'Student',
 'Taurus',
 'Technology',
 'Virgo',
 'female',
 'indUnk',
 'male']

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

### For Testing - 

### created a pipeline for steps 8 & 9 by choosing OneVsRestClassifier and Linear SVC

### trained the model on every label and computed the accuracy scores, F1, Recall & Precision scores.

In [0]:
classifier = Pipeline([
('vectorizer', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf', OneVsRestClassifier(LinearSVC()))])

classifier.fit(x_train, y_train_mlb)
predicted = classifier.predict(x_test)
print(predicted)

print("Accuracy Score using OvR and Linear SVC: ",accuracy_score(y_test_mlb, predicted))
print("F1 using OvR and Linear SVC: " + str(f1_score(y_test_mlb,predicted,average='micro')))
print("Recall using OvR and Linear SVC: " + str(recall_score(y_test_mlb,predicted,average='micro')))
print("Precision using OvR and Linear SVC: " + str(average_precision_score(y_test_mlb, predicted,average='micro')))

[[0 0 0 ... 1 0 0]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]
Accuracy Score using OvR and Linear SVC:  0.546639231824417
F1 using OvR and Linear SVC: 0.7675070028011204
Recall using OvR and Linear SVC: 0.681241426611797
Precision using OvR and Linear SVC: 0.6222784544115134


### 8. Choose a classifier



In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
clf= LogisticRegression(solver='lbfgs')
clf= OneVsRestClassifier(clf)  

### 9. Fit the classifier, make predictions and get the accuracy

In [0]:
clf.fit(x_train_ct,y_train_mlb)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [0]:
y_pred=clf.predict(x_test_ct)
y_pred

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [0]:
y_pred.shape

(1458, 54)

In [0]:
y_pred_ = mlb.inverse_transform(y_pred)

In [0]:
print(y_pred_)

[('25', 'Aquarius', 'female'), ('female', 'indUnk'), ('female',), ('35', 'Aries', 'Technology', 'male'), ('male',), ('35', 'Aries', 'Technology', 'male'), ('34', 'Sagittarius', 'female', 'indUnk'), ('35', 'Aries', 'Technology', 'male'), ('male',), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('24', 'Engineering', 'Libra', 'male'), ('35', 'Aries', 'Technology', 'male'), ('male',), ('23', '25', 'Libra', 'female', 'indUnk'), ('35', 'Aries', 'Technology', 'male'), ('male',), ('35', 'Aries', 'Technology', 'male'), ('Libra', 'Student', 'female'), ('Aries', 'male'), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('34', 'Sagittarius', 'female', 'indUnk'), ('35', 'Aries', 'Technology', 'male'), ('34', 'Sagittarius', 'female', 'indUnk'), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('female',), ('female', 'indUnk'), ('17', 'Capricorn', 'Sports-Recreation', 'fema

In [0]:
y_test_mlb.shape

(1458, 54)

In [0]:
y_test_ = mlb.inverse_transform(y_test_mlb)

In [0]:
print(y_test_)

[('25', 'Aries', 'BusinessServices', 'female'), ('34', 'Sagittarius', 'female', 'indUnk'), ('15', 'Cancer', 'Student', 'female'), ('35', 'Aries', 'Technology', 'male'), ('39', 'Communications-Media', 'Libra', 'male'), ('35', 'Aries', 'Technology', 'male'), ('34', 'Sagittarius', 'female', 'indUnk'), ('14', 'Pisces', 'Student', 'male'), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('24', 'Scorpio', 'female', 'indUnk'), ('35', 'Aries', 'Technology', 'male'), ('24', 'Engineering', 'Libra', 'male'), ('35', 'Aries', 'Technology', 'male'), ('23', 'Sagittarius', 'indUnk', 'male'), ('25', 'Libra', 'female', 'indUnk'), ('35', 'Aries', 'Technology', 'male'), ('17', 'Leo', 'Student', 'female'), ('35', 'Aries', 'Technology', 'male'), ('15', 'Libra', 'Student', 'female'), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('35', 'Aries', 'Technology', 'male'), ('34', 'Sagittarius', 'female', 'indUnk'), ('35', 'Aries', 'Technology', 'male'),

In [0]:
from sklearn import metrics
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [0]:
print("F1: " + str(f1_score(y_test_mlb,y_pred,average='micro')))
print("Recall: " + str(recall_score(y_test_mlb,y_pred,average='micro')))
print("Precision: " + str(average_precision_score(y_test_mlb, y_pred,average='micro')))
print("Accuracy:" + str(metrics.accuracy_score(y_test_mlb,y_pred))) 

F1: 0.7473664409330324
Recall: 0.681241426611797
Precision: 0.5874809519953768
Accuracy:0.5500685871056241


### 10. Print true label and predicted label for any five examples


In [0]:
y_test[0:350]

3975      [female, 25, BusinessServices, Aries]
4307          [female, 34, indUnk, Sagittarius]
3789              [female, 15, Student, Cancer]
2682              [male, 35, Technology, Aries]
3707    [male, 39, Communications-Media, Libra]
                         ...                   
2115              [male, 35, Technology, Aries]
1494              [male, 35, Technology, Aries]
2851              [male, 35, Technology, Aries]
181             [female, 33, Banking, Aquarius]
771             [male, 23, indUnk, Sagittarius]
Name: label1, Length: 350, dtype: object

In [0]:
y_test[4307]

['female', '34', 'indUnk', 'Sagittarius']

In [0]:
y_test[2682]

['male', '35', 'Technology', 'Aries']

In [0]:
y_pred_[15]

('23', '25', 'Libra', 'female', 'indUnk')

In [0]:
y_pred_[8]

('male',)

In [0]:
y_pred_[35]

('34', 'Sagittarius', 'Student', 'female', 'indUnk')

In [0]:
y_pred_[10]

('35', 'Aries', 'Technology', 'male')

In [0]:
y_test_[15]

('25', 'Libra', 'female', 'indUnk')

In [0]:
y_pred_[15]

('23', '25', 'Libra', 'female', 'indUnk')

In [0]:
y_test_[25]

('34', 'Sagittarius', 'female', 'indUnk')

In [0]:
y_test[3476]

['male', '35', 'Technology', 'Aries']

In [0]:
y_pred_[10]

('35', 'Aries', 'Technology', 'male')